In [1]:
from llm_bases.chatglm6b import ChatGML6B
glm = ChatGML6B()

/home/zf/anaconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 8/8 [00:16<00:00,  2.09s/it]


In [25]:
import sys
del sys.modules["split_llm.glm6b.wrapped_layer"]

In [26]:
from split_llm.glm6b.wrapped_layer import Attention_GLM_Wrapped, FeedForward_GLM_Wrapped, copy_attention, copy_feedforward

In [27]:
transformer_layer = glm.condgen.transformer.layers[0].float()
attn_wrapped = Attention_GLM_Wrapped(4096, 32, 0)
feedforward_wrapped = FeedForward_GLM_Wrapped(4096, 32, 0)
copy_attention(transformer_layer, attn_wrapped)
copy_feedforward(transformer_layer, None, feedforward_wrapped)

In [28]:
import torch
from split_llm.glm6b.utils import generate_attention_mask, generate_position_ids

In [29]:
xs = torch.rand([10, 1, 4096]).to("cuda:1")
transformer_layer.to("cuda:1")
attn_wrapped.to("cuda:1")
feedforward_wrapped.to("cuda:1")
position_ids = generate_position_ids(10, 10).to("cuda:1")
attention_mask = generate_attention_mask(10, 10).to("cuda:1")

In [30]:
output_raw = transformer_layer(xs, position_ids, attention_mask, 0)[0]
print(output_raw.shape)
print(output_raw)

Attention out: tensor([[[ 0.5028, -0.6945,  0.2699,  ..., -0.1979, -0.2703,  0.2806]],

        [[ 0.3037, -0.2226, -0.0655,  ..., -0.3376, -0.0304,  0.0626]],

        [[ 0.3981, -0.7152,  0.5960,  ..., -0.2712, -0.1069,  0.0912]],

        ...,

        [[ 0.2937, -0.5033,  0.5714,  ..., -0.1932, -0.3533,  0.4047]],

        [[ 0.2462, -0.4652,  0.0992,  ..., -0.3007, -0.2844,  0.2734]],

        [[ 0.3354, -0.2581, -0.1448,  ..., -0.0564,  0.0175,  0.2254]]],
       device='cuda:1') torch.Size([10, 1, 4096])
torch.Size([10, 1, 4096])
tensor([[[  2.0890,  12.7236,   8.3877,  ...,   1.5867,  -0.5379,  -2.8155]],

        [[ -9.3588,   9.4922,  -8.6534,  ..., -11.6280,  -7.5904,  -6.9502]],

        [[ -5.7781,   7.6731,  -8.7551,  ...,  -1.0469, -11.9426,   5.5981]],

        ...,

        [[ -4.2457,  12.9362, -10.2315,  ...,   7.5768,  -9.7794,   0.3820]],

        [[  1.0435,  12.2478,   1.9428,  ...,  -3.5932,  -4.4942, -10.6123]],

        [[ 12.8683,  -5.8997, -13.5504,  ...,   

In [33]:
# Compute the output by wrapped layers
xs_normalized = transformer_layer.input_layernorm(xs)
attn_out_raw = attn_wrapped(xs_normalized, position_ids)
attn_out = attn_out_raw + (2 * 28) ** 0.5 * xs_normalized
output_wrapped = feedforward_wrapped(attn_out)


Attnetion_out: tensor([[[ 0.5028, -0.6945,  0.2699,  ..., -0.1979, -0.2703,  0.2806]],

        [[ 0.3037, -0.2226, -0.0655,  ..., -0.3376, -0.0305,  0.0626]],

        [[ 0.3981, -0.7153,  0.5961,  ..., -0.2713, -0.1069,  0.0912]],

        ...,

        [[ 0.2936, -0.5032,  0.5714,  ..., -0.1932, -0.3533,  0.4046]],

        [[ 0.2462, -0.4652,  0.0991,  ..., -0.3007, -0.2844,  0.2734]],

        [[ 0.3354, -0.2582, -0.1448,  ..., -0.0562,  0.0176,  0.2253]]],
       device='cuda:1', grad_fn=<ViewBackward0>)


In [34]:
print(output_wrapped.shape)
print(output_wrapped)

torch.Size([10, 1, 4096])
tensor([[[  2.0893,  12.7238,   8.3878,  ...,   1.5862,  -0.5378,  -2.8158]],

        [[ -9.3590,   9.4928,  -8.6537,  ..., -11.6288,  -7.5909,  -6.9505]],

        [[ -5.7778,   7.6735,  -8.7553,  ...,  -1.0473, -11.9425,   5.5979]],

        ...,

        [[ -4.2454,  12.9363, -10.2317,  ...,   7.5766,  -9.7795,   0.3815]],

        [[  1.0439,  12.2479,   1.9431,  ...,  -3.5935,  -4.4943, -10.6123]],

        [[ 12.8685,  -5.8996, -13.5512,  ...,   3.8911,  -0.0961,  -5.0699]]],
       device='cuda:1', grad_fn=<AddBackward0>)


In [41]:
print("Attention input L2 norm", torch.sqrt(torch.mean(torch.square(xs_normalized))).item() * (2 * 28) ** 0.5)
print("Attention output L2 norm", torch.sqrt(torch.mean(torch.square(attn_out_raw))).item())

Attention input L2 norm 7.402734913711181
Attention output L2 norm 0.4633835256099701


It seems that the residual connected attention input has a larger scale ($\approx 20\times$) than the attention ouptut, which means that the adversary could be easy to distinguish between them?

Specifically, suppose the attention is to be protected. 

Each time we feed $H, H'$ to attention but only use the true $H$ to produce the output $a$, then it could be easy to distinguish between $(H, H + a)$ and $(H, H' + a)$...